In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# LLMs: Get predictions from a language model

LangChain의 가장 기본적인 구성 요소는 입력값에 대해 LLM을 호출하는 것입니다. 이를 수행하는 간단한 예제를 살펴보겠습니다. 이를 위해, 회사가 무엇을 생산하는지에 기반하여 회사 이름을 생성하는 서비스를 구축한다고 가정해 봅시다. 이를 위해서는 먼저 LLM 래퍼를 가져와야 합니다.

In [2]:
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0.9)

In [4]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



Bright Footwear.


# Prompt Templates: Manage prompts for LLMS

일반적으로, LLM을 사용할 때, 사용자 입력을 직접 보내는 것이 아니라, 사용자 입력을 받아 프롬프트(prompt)를 작성한 후 LLM에게 보냅니다. 이 문서에서는 LangChain을 사용하여 프롬프트 템플릿을 정의하는 방법을 설명합니다. 프롬프트 템플릿은 사용자 입력에서 필요한 정보만 추출하여 해당 정보를 이용해 프롬프트를 구성하는 것이 가능합니다.

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [6]:
print(prompt.format(product="colorful socks"))

What is a good name for a company that makes colorful socks?


# Chains: Combine LLMs and prompts in multi-step workflows

In [7]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [8]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

In [9]:
chain.run("colorful socks")

'\n\nRainbow Toe Socks'

# Agents: Dynamically Call Chains Based on User Input

에이전트는 LLM(언어 모델)을 사용하여 어떤 작업을 수행하고 그 결과를 반환하는 도구입니다. 이 때, 도구는 구글 검색, 데이터베이스 조회, Python REPL, 다른 체인 등과 같이 특정한 작업을 수행하는 함수를 의미합니다. 

LLM은 에이전트의 핵심인 언어 모델을 의미합니다. 

에이전트는 지원하는 에이전트 클래스를 참조하는 문자열입니다. 이 노트북에서는 가장 간단하고 Highest Level API를 사용하여 기본적으로 사용하는 방법에 대해 설명합니다.

In [10]:
from serpapi import GoogleSearch

GoogleSearch.SERP_API_KEY = os.getenv('SERPAPI_API_KEY')

In [11]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 51.98ºf @12:56 AM Low: 50ºf @4:45 AM Approx. Precipitation / Rain Total: 0.063 in. 1hr. Precip / Rain Total (in.)
Thought: I need to use the calculator to raise the temperature to the .023 power.
Action: Calculator
Action Input: 51.98^.023
Observation: Answer: 1.0951263641624882
Thought: